In [1]:
import text_processing
import torch
import torch.nn.functional as F
from imp import reload
import models
reload(models)
from text_processing import read_split_file_lyrics, BatchWrapper, generate_iterators_lyrics
from functions import *
import numpy as np
import pandas as pd
from models import BiLSTMSourceNet

In [2]:
print("Reading in the cleaned lyrics data.")

#Read and split data
train_path, val_path = read_split_file_lyrics("lyrics_cleaned.csv.gz","")

print("Creating iterators.")
#Create iterators
torch.cuda.empty_cache() 
batch_size = 10
train_iter, val_iter, TEXT = generate_iterators_lyrics(train_path, val_path, 
                                                           batch_size = batch_size, device="cuda")


Reading in the cleaned lyrics data.
Creating iterators.


In [14]:
dropout = 0.350748
learning_rate = 0.000146
hidden_dim = 183
v_dim = 21
label_size = 2
embedding_dim = 300

train_batch = BatchWrapper(train_iter, "lyrics", ["pos", "neg"])
valid_batch = BatchWrapper(val_iter, "lyrics", ["pos", "neg"])

m_source = BiLSTMSourceNet(vocab_size = len(TEXT.vocab), embedding_dim = embedding_dim, 
                               hidden_dim = hidden_dim, label_size=label_size, v_dim = v_dim, 
                               pretrained_vec=TEXT.vocab.vectors, use_gpu = True, dropout = dropout)
m_source.to("cuda")
opt = torch.optim.Adam(filter(lambda p: p.requires_grad, m_source.parameters()), learning_rate)

dev_acc, src_attention = training_loop_DATN(model=m_source, training_iter=train_batch, 
                                                      dev_iter=valid_batch, loss_=F.kl_div, optim=opt, num_epochs=10, 
                                                      source_flag = True, batch_size = batch_size, verbose = True)



Training...
Evaluating dev...
Epoch 0; Loss 0.278959; Dev loss 0.301836, Dev acc: 0.663542
Training...
Evaluating dev...
Epoch 1; Loss 0.218941; Dev loss 0.274055, Dev acc: 0.709509
Training...
Evaluating dev...
Epoch 2; Loss 0.212264; Dev loss 0.273205, Dev acc: 0.710417
Training...
Evaluating dev...
Epoch 3; Loss 0.289682; Dev loss 0.268850, Dev acc: 0.739812
Training...
Evaluating dev...
Epoch 4; Loss 0.191176; Dev loss 0.279378, Dev acc: 0.723270
Training...
Evaluating dev...
Epoch 5; Loss 0.176935; Dev loss 0.258766, Dev acc: 0.746875
Training...
Evaluating dev...
Epoch 6; Loss 0.153982; Dev loss 0.278248, Dev acc: 0.753396
Training...
Evaluating dev...
Epoch 7; Loss 0.198509; Dev loss 0.282652, Dev acc: 0.733542
Training...
Evaluating dev...
Epoch 8; Loss 0.363775; Dev loss 0.262311, Dev acc: 0.748428
Training...
Evaluating dev...
Epoch 9; Loss 0.458884; Dev loss 0.264616, Dev acc: 0.741667
Training...
Evaluating dev...
Epoch 10; Loss 0.263062; Dev loss 0.307349, Dev acc: 0.74528

PermissionError: [Errno 13] Permission denied: '/SourceNetModelDict_Final'

In [15]:
torch.save(m_source.state_dict(), "SourceNetModelDict_Final.pth")